In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import elasticite as el
import numpy as np
duration = el.get_default_args(el.EdgeGrid.render)['duration']

In [8]:
import sys
sys.path.append('..')

In [9]:
from scenario_line_fresnel import EdgeGrid
e = EdgeGrid(N_lame=25, grid_type='line')
e.render(fps=50, fname='../files/fresnel.mp4')

In [10]:
from scenario_line_elastic_fresnel import EdgeGrid
e = EdgeGrid(N_lame=25, grid_type='line')
e.render(fps=50., duration=10, fname='../files/elastic_fresnel.mp4')

0 0.12 0.042
1 0.12 0.042
2 0.12 0.042
3 0.12 0.042
4 0.12 0.042
5 0.12 0.042
6 0.12 0.042
7 0.12 0.042
8 0.12 0.042
9 0.12 0.042
10 0.12 0.042
11 0.12 0.042
12 0.12 0.042
13 0.12 0.042
14 0.12 0.042
15 0.12 0.042
16 0.12 0.042
17 0.12 0.042
18 0.12 0.042
19 0.12 0.042
20 0.12 0.042
21 0.12 0.042
22 0.12 0.042
23 0.12 0.042
24 0.12 0.042


FileNotFoundError: [Errno 2] No such file or directory: 'povray'

## controls

Le point focal bouge à gauche de l'observateur.

In [ ]:
import elasticite as el
import numpy as np

duration = el.get_default_args(el.EdgeGrid.render)['duration']
location = el.get_default_args(el.EdgeGrid.render)['location']


class EdgeGrid(el.EdgeGrid):
    def champ(self):
        tf = 2. #how many cycles in one period (duration)
        force = np.zeros_like(self.lames[2, :])
        damp = lambda t: 0.01 #* np.exp(np.cos(t / 6.) / 3.**2)
        xf = lambda t: location[0] - self.total_width/4.
        zf = lambda t: location[2] * (.5 + .45 * np.sin(2*np.pi*self.t*tf/duration))
        
        desired_angle = np.pi/2 + np.arctan2(self.lames[1, :]-zf(self.t), self.lames[0, :]-xf(self.t))
        self.lames[2, :] = np.mod(self.lames[2, :]-np.pi/2, np.pi) + np.pi/2
        force += np.mod(desired_angle-np.pi/2, np.pi) + np.pi/2- self.lames[2, :]
        force -= damp(self.t) * self.lames[3, :]/self.dt
        return 3. * force

e = EdgeGrid(N_lame=25, grid_type='line')
e.render(fps=50., duration=10, fname='../files/elastic_fresnel_gauche.mp4')

Lateral movement from right to left

In [ ]:
class EdgeGrid(el.EdgeGrid):
    def champ(self):
        tf = 2. #how many cycles in one period (duration)
        force = np.zeros_like(self.lames[2, :])
        damp = lambda t: 0.01 #* np.exp(np.cos(t / 6.) / 3.**2)
        xf = lambda t: location[0] + 2. * np.sin(2*np.pi*self.t*tf/duration)
        zf = lambda t: location[2]
        
        desired_angle = np.pi/2 + np.arctan2(self.lames[1, :]-zf(self.t), self.lames[0, :]-xf(self.t))
        self.lames[2, :] = np.mod(self.lames[2, :]-np.pi/2, np.pi) + np.pi/2
        force += np.mod(desired_angle-np.pi/2, np.pi) + np.pi/2- self.lames[2, :]
        force -= damp(self.t) * self.lames[3, :]/self.dt
        return 3. * force

e = EdgeGrid(N_lame=25, grid_type='line')
e.render(fps=50., duration=10, fname='../files/elastic_fresnel_lateral.mp4')

Circular movement

In [ ]:
class EdgeGrid(el.EdgeGrid):
    def champ(self):
        tf = 2. #how many cycles in one period (duration)
        force = np.zeros_like(self.lames[2, :])
        damp = lambda t: 0.01 #* np.exp(np.cos(t / 6.) / 3.**2)
        xf = lambda t: location[0] + 2. * np.sin(2*np.pi*self.t*tf/duration)
        zf = lambda t: location[2] * (.5 + .45 * np.sin(2*np.pi*self.t*tf/duration))
        
        desired_angle = np.pi/2 + np.arctan2(self.lames[1, :]-zf(self.t), self.lames[0, :]-xf(self.t))
        self.lames[2, :] = np.mod(self.lames[2, :]-np.pi/2, np.pi) + np.pi/2
        force += np.mod(desired_angle-np.pi/2, np.pi) + np.pi/2- self.lames[2, :]
        force -= damp(self.t) * self.lames[3, :]/self.dt
        return 3. * force

e = EdgeGrid(N_lame=25, grid_type='line')
e.render(fps=50., duration=10, fname='../files/elastic_fresnel_rot.mp4')

Simulation des pas du moteur

In [ ]:
class EdgeGrid(el.EdgeGrid):
    def champ(self):
        tf = 2. #how many cycles in one period (duration)
        force = np.zeros_like(self.lames[2, :])
        damp = lambda t: 0.01 #* np.exp(np.cos(t / 6.) / 3.**2)
        xf = lambda t: location[0] #+ 2. * np.sin(2*np.pi*self.t*tf/duration)
        zf = lambda t: location[2] * (.5 + .45 * np.sin(2*np.pi*self.t*tf/duration))
        
        desired_angle = np.pi/2 + np.arctan2(self.lames[1, :]-zf(self.t), self.lames[0, :]-xf(self.t))
        self.lames[2, :] = np.mod(self.lames[2, :]-np.pi/2, np.pi) + np.pi/2
        force += np.mod(desired_angle-np.pi/2, np.pi) + np.pi/2- self.lames[2, :]
        force -= damp(self.t) * self.lames[3, :]/self.dt
        return 3. * force

    def update(self):
        self.lames[2, :] += self.lames[3, :]*self.dt/2
        self.lames[3, :] += self.champ() * self.dt
        self.lames[2, :] += self.lames[3, :]*self.dt/2
        self.lames[2, :] = np.floor(self.lames[2, :] / np.pi* self.n_pas ) / self.n_pas * np.pi

e = EdgeGrid(N_lame=25, grid_type='line')
e.n_pas = 150.
e.render(fps=50., duration=10, fname='../files/elastic_fresnel_pasapas.mp4')